In [ ]:
import pandas as pd
import numpy as np
import itertools as it
import os
import io
import logging
import boto3
import sys
import datetime
from scipy.special import logit, expit
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, RandomizedSearchCV
from xgboost.sklearn import XGBRegressor


In [ ]:
TARGET_DATE: str = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [ ]:
!pip freeze

# Reading Data 

Reading data from multiple sources, with the corresponding query

### Step 1 of the Prediction Process: Getting Data
Step 1.1: update the funnel metrics by Sagemaker ipynb file 'query_pipeline' under the '/query' folder

Step 1.2: run each of the query in the '/day28_prediction/query/' to extract each input csv below

In [ ]:
logger = logging.getLogger()
logger.info(f'Loading inputs')

s3 = boto3.resource('s3')
bucket = s3.Bucket(input_bucket)

data_list = []
# Iterates through all the objects, doing the pagination for you. Each obj
# is an ObjectSummary, so it doesn't contain the body. You'll need to call
# get to get the whole body.
for obj in bucket.objects.filter(Prefix='hourspct'):
    key = obj.key
    if 'sagemaker' not in key:
        logger.info('Loading csv file {}'.format(key))
        body = obj.get()['Body']
        var_name = key.split('.')[0].split('/')[1]
        print('Reading {0} features'.format(var_name))
        exec("{0}=pd.read_csv(body, na_values = [r'\\\\N'])".format(var_name))
        exec("{0}.columns = {0}.columns.str.lower()".format(var_name))
        
        # exclude the full null columns
        exec("{0} = {0}.loc[:,{0}.isnull().sum()!={0}.shape[0]]".format(var_name))
        
        # append the feature df
        exec("data_list.append({0})".format(var_name))

In [ ]:
for cnt in range(len(data_list)):
    data_list[cnt].columns = data_list[cnt].columns.str.lower().str.replace("'", "")

df = data_list[0]

for data in data_list[1:]:
    df = df.merge(data, how = 'outer', on = 'match_id')

df.columns = df.columns.str.lower()
df = df.loc[~df.match_id.duplicated()]
df = df.set_index('match_id')

In [ ]:
# only use shows with at least 14d history
df = df.loc[~(df.hourspct14d.isnull() | df.hourspct7d.isnull())]
df['max_window'] = 14
df.loc[~df.hourspct21d.isnull(), 'max_window'] = 21
df.loc[~df.hourspct28d.isnull(), 'max_window'] = 28
df.loc[~df.hourspct8w.isnull(), 'max_window'] = 56
df.loc[~df.hourspct13w.isnull(), 'max_window'] = 91
df.loc[~df.hourspct26w.isnull(), 'max_window'] = 182

df = df.drop(columns=['licensor_agg',
                     'match_id_platform',
                     'navigation_genre_desc_agg',
                     'descriptive_genre_desc_agg',
                     'title_name']
            ,errors='ignore')

df['wm_enterprise_genres_agg'] = df.wm_enterprise_genres_agg.fillna('nan').apply(lambda x: x.replace(' |', '|').replace('| ', '|'))
df['launch_time'] = (pd.to_datetime(df['earliest_offered_timestamp']) - pd.to_datetime('2020-05-27')).dt.days


In [ ]:
df['logit_hourspct7d'] = logit(df['hourspct7d']/100.0)
df['logit_hourspct14d'] = logit(df['hourspct14d']/100.0)
df['logit_hourspct21d'] = logit(df['hourspct21d']/100.0)
df['logit_hourspct28d'] = logit(df['hourspct28d']/100.0)
df['logit_hourspct8w'] = logit(df['hourspct8w']/100.0)
df['logit_hourspct13w'] = logit(df['hourspct13w']/100.0)
df['logit_hourspct26w'] = logit(df['hourspct26w']/100.0)
df['logit_hourspct52w'] = logit(df['hourspct52w']/100.0)

In [ ]:
glist = set()
for item in list(df.wm_enterprise_genres_agg):
    glist = glist | set(item.split('|'))
if '' in glist:
    glist.remove('')
genre_cols = pd.DataFrame(index=df.index, columns=glist)


for match_id in df.index:
    genres = df.loc[match_id, 'wm_enterprise_genres_agg'].split('|')
    if '' in genres:
        genres.remove('')
    genre_cols.loc[match_id, genres] = 1
    
genre_cols = genre_cols.add_prefix('genre_').fillna(0)
df = pd.concat([df, genre_cols], axis=1).drop(columns='wm_enterprise_genres_agg')

In [ ]:
df.loc[df.program_type=='unknown', 'program_type'] = np.nan
df.loc[df.program_type=='acquired', 'program_type'] = 0
df.loc[df.program_type=='original', 'program_type'] = 1

#df.loc[df.content_category=='special', 'content_category'] = 0
#df.loc[df.content_category=='movies', 'content_category'] = 0
df.loc[df.content_category!='series', 'content_category'] = 0
df.loc[df.content_category=='series', 'content_category'] = 1


df = df.rename(columns={'program_type': 'is_original', 'content_category': 'is_series'})
df.is_original = df.is_original.astype('float')
df.is_series = df.is_series.astype('float')

In [ ]:
df_fulldata = df.loc[~df.hourspct52w.isnull()]
df_fulldata = df_fulldata.drop(df_fulldata.loc[df_fulldata.hourspct52w==0].index)
logger.info(f'df columns: {list(df_fulldata.columns)}')

In [ ]:
# random cutting of hourspct values for model to learn null values
# 10% set as null

# only works in numpy >= 1.17
#rng = np.random.default_rng(15)
#df_mask = pd.DataFrame(rng.choice([0,np.nan], size=[len(df_fulldata),5], p=[0.9,0.1]),

np.random.seed(242)
df_mask = pd.DataFrame(np.random.choice([0,np.nan], size=[len(df_fulldata),5], p=[0.9,0.1]),
                       columns=['logit_hourspct21d',
                                'logit_hourspct28d',
                                'logit_hourspct8w',
                                'logit_hourspct13w',
                                'logit_hourspct26w'],
                       index=df_fulldata.index)

df_fulldata.loc[:,['logit_hourspct21d','logit_hourspct28d','logit_hourspct8w','logit_hourspct13w','logit_hourspct26w']] += df_mask

In [ ]:
xcols = ['logit_hourspct7d',
         'logit_hourspct14d',
         'logit_hourspct21d',
         'logit_hourspct28d',
         'logit_hourspct8w',
         'logit_hourspct13w',
         'logit_hourspct26w',
         'season_number_adj',
         'is_original',
         'is_series',
         'single_episode_ind',
         'in_sequantial_releasing_period',
         'at_release_year',
         'dayofweek_earliest_date',
         'total_hours',
         'prod_release_year',
         'title_age_approx',
         'content_cost',
         'ln_total_media_cost_pre_launch',
         'min_days_since_offered',
         'launch_time',
         'genre_sports',
         'genre_romance',
         'genre_news/talk',
         'genre_music',
         'genre_drama',
         'genre_crime',
         'genre_latino',
         'genre_originals',
         'genre_suspense',
         'genre_comedy',
         'genre_documentary',
         'genre_action',
         'genre_international',
         'genre_reality',
         'genre_nan',
         'genre_fantasy & sci-fi',
         'genre_shorts',
         'genre_horror'
        ]
ycol = ['logit_hourspct52w']
trainx,testx,trainy,testy = train_test_split(df_fulldata[xcols], df_fulldata[ycol], test_size=0.2, random_state=58)

In [1]:
param = {'objective': 'reg:squarederror'
         , 'n_estimators': 200
         , 'min_child_weight': 5
         , 'max_depth': 5
         , 'learning_rate': 0.3
         , 'gamma': 0.3
         , 'colsample_bytree': 1.0}

gridspace = {'objective': [ 'reg:squarederror' ]
             ,"learning_rate" : [ 0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ]
             ,"max_depth" : [ 2, 3, 4, 5, 6, 8, 10 ]
             , "min_child_weight" : [ 1, 3, 5, 7 ]
             , "gamma" : [ 0.0, 0.1, 0.2 , 0.3, 0.4, 0.5 ]
             , "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7, 1.0 ] }
testgrid = {
    "max_depth": [2,3]
    ,'objective': ['reg:squarederror']
}

In [ ]:
xgbr = XGBRegressor()

xgb_grid = RandomizedSearchCV(xgbr,
                        gridspace,
                        cv = 10,
                        n_iter = 100,
                        n_jobs = -1,
                        verbose=True)

xgb_grid.fit(trainx, trainy)

In [ ]:
pred_52w = expit(xgb_grid.best_estimator_.predict(df.loc[df.hourspct52w.isnull(), xcols]))*100

df_pred_52w = pd.DataFrame(pred_52w, index = df.loc[df.hourspct52w.isnull()].index)
df_pred_52w.columns = ['predicted_hourspct']
#TODO df formatting for table

df_pred_52w['prediction_date'] = TARGET_DATE
df_pred_52w['window_days'] = df.loc[df_pred_52w.index, 'max_window']
df_pred_52w = df_pred_52w.reset_index()

# Write csvs to S3

### Step 3: Write the prediction result to S3

In [ ]:
def to_s3(filename, output_bucket, content):
    client = boto3.client('s3')
    client.put_object(Bucket=output_bucket, Key='hourspct/'+filename, Body=content)

In [ ]:
logger.info('Writing hours% predictions to S3 as an csv file')
print('Writing hours% predictions to S3 as an csv file')
csv_buffer = io.StringIO()
df_pred_52w.to_csv(csv_buffer, index = False)
content = csv_buffer.getvalue()

filename = 'new_hourspct_prediction.csv'

to_s3(filename, output_bucket, content)